# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [42]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [43]:
#load the csv
weather_df = "output_data/weather_data.csv"
weather_df = pd.read_csv(weather_df)
weather_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country
0,0,Ixtapa,20.70,-105.20,80.29,83,90,8.05,1599964578,MX
1,1,Synya,65.37,58.04,42.03,93,100,9.37,1599964579,RU
2,2,Albany,42.60,-73.97,57.22,99,0,8.21,1599964375,US
3,3,Jamestown,42.10,-79.24,64.35,88,90,12.75,1599964579,US
4,4,Clyde River,70.47,-68.59,33.80,83,90,4.70,1599964580,CA


In [ ]:
### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [46]:
#config gmaps
gmaps.configure(api_key=g_key)

#store the lat and long
locations = weather_df[["Latitude", "Longitude"]]

#store humidity in humidity
humidity = weather_df["Humidity"]

In [47]:
# create a poverty Heatmap layer
fig = gmaps.figure(center=(48.0, 0.0, zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 1, opacity=1)
 
fig.add_layer(heat_layer)
plt.savefig("Vacation_Heat_Map.png")
fig

SyntaxError: invalid syntax (<ipython-input-47-b24874176de6>, line 2)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
